In [1]:
!pip install keras-tuner

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
    

print(tf.__version__)

2.1.0


In [30]:
import pandas as pd
# Load the data
#train = pd.read_csv("../input/train.csv")
#test = pd.read_csv("../input/test.csv")

train = pd.read_csv("digit-recognizer/train.csv")
test = pd.read_csv("digit-recognizer/test.csv")

In [31]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
X_train=train.drop(labels=['label'],axis = 1)
y_train=train['label']

In [33]:
X_train.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
test.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Check for null and missing values

In [35]:
X_train.isnull().any().describe()

count       784
unique        1
top       False
freq        784
dtype: object

# Normalization

In [36]:
# Normalize the data between 0 to 1 to improve the processing 
X_train = X_train / 255.0
test = test / 255.0

In [37]:
X_train.shape

(42000, 784)

Train and test images (28px x 28px) has been stock into pandas.Dataframe as 1D vectors of 784 values. We reshape all data to 28x28x1 3D matrices.

Keras requires an extra dimension in the end which correspond to channels. MNIST images are gray scaled so it use only one channel. For RGB images, there is 3 channels, we would have reshaped 784px vectors to 28x28x3 3D matrices.

In [38]:
X_train=X_train.values.reshape(len(X_train),28,28,1)
test=test.values.reshape(len(test),28,28,1)

# CNN Optimizer using hyperparameter

In [39]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [40]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [41]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Digit Recognizer")

In [43]:
tuner_search.search(X_train,y_train,epochs=3,validation_split=0.1)

Train on 37800 samples, validate on 4200 samples
Epoch 1/3


 5888/37800 [===>..........................] - ETA: 1:08:41 - loss: 2.3130 - accuracy: 0.031 - ETA: 36:21 - loss: 2.2991 - accuracy: 0.0938  - ETA: 25:12 - loss: 2.2709 - accuracy: 0.187 - ETA: 19:43 - loss: 2.2482 - accuracy: 0.218 - ETA: 16:22 - loss: 2.1821 - accuracy: 0.287 - ETA: 14:09 - loss: 2.1009 - accuracy: 0.338 - ETA: 12:34 - loss: 2.0319 - accuracy: 0.379 - ETA: 11:23 - loss: 1.9798 - accuracy: 0.402 - ETA: 10:27 - loss: 1.9379 - accuracy: 0.413 - ETA: 9:42 - loss: 1.8693 - accuracy: 0.437 - ETA: 9:06 - loss: 1.8376 - accuracy: 0.44 - ETA: 8:35 - loss: 1.8242 - accuracy: 0.44 - ETA: 8:09 - loss: 1.7749 - accuracy: 0.45 - ETA: 7:46 - loss: 1.7013 - accuracy: 0.48 - ETA: 7:27 - loss: 1.6481 - accuracy: 0.50 - ETA: 7:11 - loss: 1.5987 - accuracy: 0.51 - ETA: 6:56 - loss: 1.5506 - accuracy: 0.53 - ETA: 6:43 - loss: 1.4980 - accuracy: 0.55 - ETA: 6:31 - loss: 1.4636 - accuracy: 0.56 - ETA: 6:21 - loss: 1.4338 - accuracy: 0.56 - ETA: 6:11 - loss: 1.3961 - accuracy: 0.57 - ETA: 6

17792/37800 [=============>................] - ETA: 2:17 - loss: 0.2683 - accuracy: 0.92 - ETA: 2:17 - loss: 0.2681 - accuracy: 0.92 - ETA: 2:17 - loss: 0.2677 - accuracy: 0.92 - ETA: 2:17 - loss: 0.2675 - accuracy: 0.92 - ETA: 2:16 - loss: 0.2674 - accuracy: 0.92 - ETA: 2:16 - loss: 0.2670 - accuracy: 0.92 - ETA: 2:16 - loss: 0.2666 - accuracy: 0.92 - ETA: 2:16 - loss: 0.2665 - accuracy: 0.92 - ETA: 2:16 - loss: 0.2661 - accuracy: 0.92 - ETA: 2:16 - loss: 0.2661 - accuracy: 0.92 - ETA: 2:15 - loss: 0.2660 - accuracy: 0.92 - ETA: 2:15 - loss: 0.2657 - accuracy: 0.92 - ETA: 2:15 - loss: 0.2653 - accuracy: 0.92 - ETA: 2:15 - loss: 0.2647 - accuracy: 0.92 - ETA: 2:15 - loss: 0.2640 - accuracy: 0.92 - ETA: 2:15 - loss: 0.2636 - accuracy: 0.92 - ETA: 2:14 - loss: 0.2632 - accuracy: 0.92 - ETA: 2:14 - loss: 0.2627 - accuracy: 0.92 - ETA: 2:14 - loss: 0.2621 - accuracy: 0.92 - ETA: 2:14 - loss: 0.2625 - accuracy: 0.92 - ETA: 2:14 - loss: 0.2625 - accuracy: 0.92 - ETA: 2:13 - loss: 0.2622 - ac

23744/37800 [=================>............] - ETA: 1:46 - loss: 0.2153 - accuracy: 0.93 - ETA: 1:46 - loss: 0.2150 - accuracy: 0.93 - ETA: 1:46 - loss: 0.2147 - accuracy: 0.93 - ETA: 1:45 - loss: 0.2148 - accuracy: 0.93 - ETA: 1:45 - loss: 0.2147 - accuracy: 0.93 - ETA: 1:45 - loss: 0.2144 - accuracy: 0.93 - ETA: 1:45 - loss: 0.2141 - accuracy: 0.93 - ETA: 1:45 - loss: 0.2137 - accuracy: 0.93 - ETA: 1:45 - loss: 0.2135 - accuracy: 0.93 - ETA: 1:44 - loss: 0.2132 - accuracy: 0.93 - ETA: 1:44 - loss: 0.2128 - accuracy: 0.93 - ETA: 1:44 - loss: 0.2125 - accuracy: 0.93 - ETA: 1:44 - loss: 0.2121 - accuracy: 0.93 - ETA: 1:44 - loss: 0.2118 - accuracy: 0.93 - ETA: 1:44 - loss: 0.2115 - accuracy: 0.93 - ETA: 1:44 - loss: 0.2115 - accuracy: 0.93 - ETA: 1:44 - loss: 0.2112 - accuracy: 0.93 - ETA: 1:43 - loss: 0.2112 - accuracy: 0.93 - ETA: 1:43 - loss: 0.2109 - accuracy: 0.93 - ETA: 1:43 - loss: 0.2105 - accuracy: 0.93 - ETA: 1:43 - loss: 0.2104 - accuracy: 0.93 - ETA: 1:43 - loss: 0.2100 - ac

29696/37800 [======================>.......] - ETA: 1:15 - loss: 0.1832 - accuracy: 0.94 - ETA: 1:15 - loss: 0.1831 - accuracy: 0.94 - ETA: 1:15 - loss: 0.1831 - accuracy: 0.94 - ETA: 1:15 - loss: 0.1829 - accuracy: 0.94 - ETA: 1:15 - loss: 0.1827 - accuracy: 0.94 - ETA: 1:15 - loss: 0.1826 - accuracy: 0.94 - ETA: 1:14 - loss: 0.1825 - accuracy: 0.94 - ETA: 1:14 - loss: 0.1823 - accuracy: 0.94 - ETA: 1:14 - loss: 0.1820 - accuracy: 0.94 - ETA: 1:14 - loss: 0.1821 - accuracy: 0.94 - ETA: 1:14 - loss: 0.1820 - accuracy: 0.94 - ETA: 1:14 - loss: 0.1818 - accuracy: 0.94 - ETA: 1:13 - loss: 0.1816 - accuracy: 0.94 - ETA: 1:13 - loss: 0.1813 - accuracy: 0.94 - ETA: 1:13 - loss: 0.1811 - accuracy: 0.94 - ETA: 1:13 - loss: 0.1810 - accuracy: 0.94 - ETA: 1:13 - loss: 0.1810 - accuracy: 0.94 - ETA: 1:13 - loss: 0.1808 - accuracy: 0.94 - ETA: 1:12 - loss: 0.1807 - accuracy: 0.94 - ETA: 1:12 - loss: 0.1805 - accuracy: 0.94 - ETA: 1:12 - loss: 0.1804 - accuracy: 0.94 - ETA: 1:12 - loss: 0.1801 - ac

...

23808/37800 [=================>............] - ETA: 2:09 - loss: 2.3021 - accuracy: 0.11 - ETA: 2:08 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:08 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:08 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:08 - loss: 2.3021 - accuracy: 0.11 - ETA: 2:07 - loss: 2.3021 - accuracy: 0.11 - ETA: 2:07 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:07 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:07 - loss: 2.3021 - accuracy: 0.11 - ETA: 2:07 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:06 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:06 - loss: 2.3021 - accuracy: 0.11 - ETA: 2:06 - loss: 2.3021 - accuracy: 0.11 - ETA: 2:06 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:05 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:05 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:05 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:05 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:05 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:04 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:04 - loss: 2.3022 - accuracy: 0.11 - ETA: 2:04 - loss: 2.3022 - ac

29760/37800 [======================>.......] - ETA: 1:30 - loss: 2.3025 - accuracy: 0.11 - ETA: 1:29 - loss: 2.3025 - accuracy: 0.11 - ETA: 1:29 - loss: 2.3024 - accuracy: 0.11 - ETA: 1:29 - loss: 2.3024 - accuracy: 0.11 - ETA: 1:29 - loss: 2.3024 - accuracy: 0.11 - ETA: 1:29 - loss: 2.3024 - accuracy: 0.11 - ETA: 1:28 - loss: 2.3024 - accuracy: 0.11 - ETA: 1:28 - loss: 2.3024 - accuracy: 0.11 - ETA: 1:28 - loss: 2.3024 - accuracy: 0.11 - ETA: 1:28 - loss: 2.3024 - accuracy: 0.11 - ETA: 1:28 - loss: 2.3025 - accuracy: 0.11 - ETA: 1:27 - loss: 2.3025 - accuracy: 0.11 - ETA: 1:27 - loss: 2.3025 - accuracy: 0.11 - ETA: 1:27 - loss: 2.3025 - accuracy: 0.11 - ETA: 1:27 - loss: 2.3025 - accuracy: 0.11 - ETA: 1:27 - loss: 2.3025 - accuracy: 0.11 - ETA: 1:26 - loss: 2.3025 - accuracy: 0.11 - ETA: 1:26 - loss: 2.3025 - accuracy: 0.11 - ETA: 1:26 - loss: 2.3025 - accuracy: 0.11 - ETA: 1:26 - loss: 2.3025 - accuracy: 0.11 - ETA: 1:25 - loss: 2.3025 - accuracy: 0.11 - ETA: 1:25 - loss: 2.3025 - ac

35712/37800 [===========================>..] - ETA: 51s - loss: 2.3025 - accuracy: 0.112 - ETA: 51s - loss: 2.3025 - accuracy: 0.112 - ETA: 51s - loss: 2.3025 - accuracy: 0.112 - ETA: 51s - loss: 2.3025 - accuracy: 0.112 - ETA: 50s - loss: 2.3025 - accuracy: 0.112 - ETA: 50s - loss: 2.3025 - accuracy: 0.112 - ETA: 50s - loss: 2.3025 - accuracy: 0.112 - ETA: 50s - loss: 2.3025 - accuracy: 0.112 - ETA: 50s - loss: 2.3025 - accuracy: 0.112 - ETA: 49s - loss: 2.3025 - accuracy: 0.112 - ETA: 49s - loss: 2.3025 - accuracy: 0.112 - ETA: 49s - loss: 2.3025 - accuracy: 0.112 - ETA: 49s - loss: 2.3025 - accuracy: 0.112 - ETA: 49s - loss: 2.3025 - accuracy: 0.112 - ETA: 48s - loss: 2.3025 - accuracy: 0.112 - ETA: 48s - loss: 2.3025 - accuracy: 0.112 - ETA: 48s - loss: 2.3025 - accuracy: 0.112 - ETA: 48s - loss: 2.3026 - accuracy: 0.112 - ETA: 48s - loss: 2.3026 - accuracy: 0.112 - ETA: 47s - loss: 2.3025 - accuracy: 0.112 - ETA: 47s - loss: 2.3025 - accuracy: 0.112 - ETA: 47s - loss: 2.3026 - acc

37800/37800 [==============================] - ETA: 13s - loss: 2.3026 - accuracy: 0.110 - ETA: 13s - loss: 2.3026 - accuracy: 0.110 - ETA: 12s - loss: 2.3026 - accuracy: 0.110 - ETA: 12s - loss: 2.3025 - accuracy: 0.110 - ETA: 12s - loss: 2.3025 - accuracy: 0.110 - ETA: 12s - loss: 2.3025 - accuracy: 0.110 - ETA: 12s - loss: 2.3025 - accuracy: 0.110 - ETA: 11s - loss: 2.3025 - accuracy: 0.110 - ETA: 11s - loss: 2.3025 - accuracy: 0.110 - ETA: 11s - loss: 2.3025 - accuracy: 0.110 - ETA: 11s - loss: 2.3025 - accuracy: 0.110 - ETA: 11s - loss: 2.3025 - accuracy: 0.110 - ETA: 10s - loss: 2.3025 - accuracy: 0.110 - ETA: 10s - loss: 2.3025 - accuracy: 0.110 - ETA: 10s - loss: 2.3025 - accuracy: 0.110 - ETA: 10s - loss: 2.3025 - accuracy: 0.110 - ETA: 10s - loss: 2.3025 - accuracy: 0.110 - ETA: 9s - loss: 2.3025 - accuracy: 0.110 - ETA: 9s - loss: 2.3025 - accuracy: 0.11 - ETA: 9s - loss: 2.3025 - accuracy: 0.11 - ETA: 9s - loss: 2.3025 - accuracy: 0.11 - ETA: 8s - loss: 2.3025 - accuracy: 0

INFO:tensorflow:Oracle triggered exit


In [44]:
model=tuner_search.get_best_models(num_models=1)[0]

In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 96)        960       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        115248    
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                743456    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 859,994
Trainable params: 859,994
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.fit(X_train, y_train, epochs=10, validation_split=0.1, initial_epoch=3)

Train on 37800 samples, validate on 4200 samples
Epoch 4/10


 5888/37800 [===>..........................] - ETA: 14:44 - loss: 8.8086e-04 - accuracy: 1.000 - ETA: 9:02 - loss: 0.0039 - accuracy: 1.0000    - ETA: 7:04 - loss: 0.0047 - accuracy: 1.00 - ETA: 6:07 - loss: 0.0293 - accuracy: 0.99 - ETA: 5:32 - loss: 0.0240 - accuracy: 0.99 - ETA: 5:09 - loss: 0.0216 - accuracy: 0.99 - ETA: 4:52 - loss: 0.0190 - accuracy: 0.99 - ETA: 4:40 - loss: 0.0169 - accuracy: 0.99 - ETA: 4:30 - loss: 0.0152 - accuracy: 0.99 - ETA: 4:22 - loss: 0.0137 - accuracy: 0.99 - ETA: 4:16 - loss: 0.0135 - accuracy: 0.99 - ETA: 4:11 - loss: 0.0126 - accuracy: 0.99 - ETA: 4:06 - loss: 0.0160 - accuracy: 0.99 - ETA: 4:03 - loss: 0.0151 - accuracy: 0.99 - ETA: 3:59 - loss: 0.0144 - accuracy: 0.99 - ETA: 3:56 - loss: 0.0136 - accuracy: 0.99 - ETA: 3:53 - loss: 0.0135 - accuracy: 0.99 - ETA: 3:50 - loss: 0.0128 - accuracy: 0.99 - ETA: 3:49 - loss: 0.0128 - accuracy: 0.99 - ETA: 3:46 - loss: 0.0125 - accuracy: 0.99 - ETA: 3:45 - loss: 0.0182 - accuracy: 0.99 - ETA: 3:43 - loss: 

17792/37800 [=============>................] - ETA: 2:25 - loss: 0.0150 - accuracy: 0.99 - ETA: 2:25 - loss: 0.0150 - accuracy: 0.99 - ETA: 2:25 - loss: 0.0153 - accuracy: 0.99 - ETA: 2:24 - loss: 0.0152 - accuracy: 0.99 - ETA: 2:24 - loss: 0.0154 - accuracy: 0.99 - ETA: 2:24 - loss: 0.0153 - accuracy: 0.99 - ETA: 2:24 - loss: 0.0153 - accuracy: 0.99 - ETA: 2:24 - loss: 0.0153 - accuracy: 0.99 - ETA: 2:24 - loss: 0.0153 - accuracy: 0.99 - ETA: 2:23 - loss: 0.0153 - accuracy: 0.99 - ETA: 2:23 - loss: 0.0155 - accuracy: 0.99 - ETA: 2:23 - loss: 0.0155 - accuracy: 0.99 - ETA: 2:23 - loss: 0.0155 - accuracy: 0.99 - ETA: 2:23 - loss: 0.0155 - accuracy: 0.99 - ETA: 2:23 - loss: 0.0156 - accuracy: 0.99 - ETA: 2:22 - loss: 0.0155 - accuracy: 0.99 - ETA: 2:22 - loss: 0.0155 - accuracy: 0.99 - ETA: 2:22 - loss: 0.0154 - accuracy: 0.99 - ETA: 2:22 - loss: 0.0154 - accuracy: 0.99 - ETA: 2:22 - loss: 0.0154 - accuracy: 0.99 - ETA: 2:21 - loss: 0.0153 - accuracy: 0.99 - ETA: 2:21 - loss: 0.0153 - ac

23744/37800 [=================>............] - ETA: 1:52 - loss: 0.0184 - accuracy: 0.99 - ETA: 1:52 - loss: 0.0184 - accuracy: 0.99 - ETA: 1:52 - loss: 0.0184 - accuracy: 0.99 - ETA: 1:51 - loss: 0.0183 - accuracy: 0.99 - ETA: 1:51 - loss: 0.0183 - accuracy: 0.99 - ETA: 1:51 - loss: 0.0183 - accuracy: 0.99 - ETA: 1:51 - loss: 0.0182 - accuracy: 0.99 - ETA: 1:51 - loss: 0.0182 - accuracy: 0.99 - ETA: 1:50 - loss: 0.0182 - accuracy: 0.99 - ETA: 1:50 - loss: 0.0181 - accuracy: 0.99 - ETA: 1:50 - loss: 0.0182 - accuracy: 0.99 - ETA: 1:50 - loss: 0.0182 - accuracy: 0.99 - ETA: 1:50 - loss: 0.0182 - accuracy: 0.99 - ETA: 1:49 - loss: 0.0181 - accuracy: 0.99 - ETA: 1:49 - loss: 0.0181 - accuracy: 0.99 - ETA: 1:49 - loss: 0.0181 - accuracy: 0.99 - ETA: 1:49 - loss: 0.0181 - accuracy: 0.99 - ETA: 1:49 - loss: 0.0181 - accuracy: 0.99 - ETA: 1:49 - loss: 0.0181 - accuracy: 0.99 - ETA: 1:48 - loss: 0.0181 - accuracy: 0.99 - ETA: 1:48 - loss: 0.0181 - accuracy: 0.99 - ETA: 1:48 - loss: 0.0180 - ac

29696/37800 [======================>.......] - ETA: 1:19 - loss: 0.0178 - accuracy: 0.99 - ETA: 1:18 - loss: 0.0180 - accuracy: 0.99 - ETA: 1:18 - loss: 0.0180 - accuracy: 0.99 - ETA: 1:18 - loss: 0.0180 - accuracy: 0.99 - ETA: 1:18 - loss: 0.0179 - accuracy: 0.99 - ETA: 1:18 - loss: 0.0179 - accuracy: 0.99 - ETA: 1:18 - loss: 0.0180 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0180 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0180 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0180 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0180 - accuracy: 0.99 - ETA: 1:17 - loss: 0.0180 - accuracy: 0.99 - ETA: 1:16 - loss: 0.0179 - accuracy: 0.99 - ETA: 1:16 - loss: 0.0180 - accuracy: 0.99 - ETA: 1:16 - loss: 0.0180 - accuracy: 0.99 - ETA: 1:16 - loss: 0.0180 - accuracy: 0.99 - ETA: 1:16 - loss: 0.0179 - accuracy: 0.99 - ETA: 1:15 - loss: 0.0179 - accuracy: 0.99 - ETA: 1:15 - loss: 0.0179 - accuracy: 0.99 - ETA: 1:15 - loss: 0.0179 - accuracy: 0.99 - ETA: 1:15 - loss: 0.0179 - accuracy: 0.99 - ETA: 1:15 - loss: 0.0179 - ac

...

17824/37800 [=============>................] - ETA: 2:29 - loss: 0.0022 - accuracy: 0.99 - ETA: 2:29 - loss: 0.0022 - accuracy: 0.99 - ETA: 2:29 - loss: 0.0022 - accuracy: 0.99 - ETA: 2:29 - loss: 0.0022 - accuracy: 0.99 - ETA: 2:29 - loss: 0.0022 - accuracy: 0.99 - ETA: 2:28 - loss: 0.0022 - accuracy: 0.99 - ETA: 2:28 - loss: 0.0022 - accuracy: 0.99 - ETA: 2:28 - loss: 0.0022 - accuracy: 0.99 - ETA: 2:28 - loss: 0.0022 - accuracy: 0.99 - ETA: 2:28 - loss: 0.0022 - accuracy: 0.99 - ETA: 2:28 - loss: 0.0022 - accuracy: 0.99 - ETA: 2:28 - loss: 0.0022 - accuracy: 0.99 - ETA: 2:27 - loss: 0.0022 - accuracy: 0.99 - ETA: 2:27 - loss: 0.0021 - accuracy: 0.99 - ETA: 2:27 - loss: 0.0021 - accuracy: 0.99 - ETA: 2:27 - loss: 0.0021 - accuracy: 0.99 - ETA: 2:27 - loss: 0.0021 - accuracy: 0.99 - ETA: 2:27 - loss: 0.0021 - accuracy: 0.99 - ETA: 2:27 - loss: 0.0021 - accuracy: 0.99 - ETA: 2:26 - loss: 0.0021 - accuracy: 0.99 - ETA: 2:26 - loss: 0.0021 - accuracy: 0.99 - ETA: 2:26 - loss: 0.0021 - ac

23776/37800 [=================>............] - ETA: 2:04 - loss: 0.0028 - accuracy: 0.99 - ETA: 2:04 - loss: 0.0028 - accuracy: 0.99 - ETA: 2:04 - loss: 0.0028 - accuracy: 0.99 - ETA: 2:04 - loss: 0.0028 - accuracy: 0.99 - ETA: 2:04 - loss: 0.0028 - accuracy: 0.99 - ETA: 2:03 - loss: 0.0028 - accuracy: 0.99 - ETA: 2:03 - loss: 0.0028 - accuracy: 0.99 - ETA: 2:03 - loss: 0.0028 - accuracy: 0.99 - ETA: 2:03 - loss: 0.0027 - accuracy: 0.99 - ETA: 2:03 - loss: 0.0027 - accuracy: 0.99 - ETA: 2:02 - loss: 0.0027 - accuracy: 0.99 - ETA: 2:02 - loss: 0.0027 - accuracy: 0.99 - ETA: 2:02 - loss: 0.0027 - accuracy: 0.99 - ETA: 2:02 - loss: 0.0027 - accuracy: 0.99 - ETA: 2:02 - loss: 0.0027 - accuracy: 0.99 - ETA: 2:02 - loss: 0.0027 - accuracy: 0.99 - ETA: 2:02 - loss: 0.0028 - accuracy: 0.99 - ETA: 2:02 - loss: 0.0028 - accuracy: 0.99 - ETA: 2:01 - loss: 0.0028 - accuracy: 0.99 - ETA: 2:01 - loss: 0.0028 - accuracy: 0.99 - ETA: 2:01 - loss: 0.0028 - accuracy: 0.99 - ETA: 2:01 - loss: 0.0028 - ac

29728/37800 [======================>.......] - ETA: 1:30 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:30 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:29 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:29 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:29 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:29 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:29 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:29 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:28 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:28 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:28 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:28 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:28 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:28 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:27 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:27 - loss: 0.0031 - accuracy: 0.99 - ETA: 1:27 - loss: 0.0032 - accuracy: 0.99 - ETA: 1:27 - loss: 0.0032 - accuracy: 0.99 - ETA: 1:27 - loss: 0.0032 - accuracy: 0.99 - ETA: 1:26 - loss: 0.0032 - accuracy: 0.99 - ETA: 1:26 - loss: 0.0032 - accuracy: 0.99 - ETA: 1:26 - loss: 0.0032 - ac

35680/37800 [===========================>..] - ETA: 54s - loss: 0.0044 - accuracy: 0.998 - ETA: 54s - loss: 0.0044 - accuracy: 0.998 - ETA: 53s - loss: 0.0044 - accuracy: 0.998 - ETA: 53s - loss: 0.0044 - accuracy: 0.998 - ETA: 53s - loss: 0.0044 - accuracy: 0.998 - ETA: 53s - loss: 0.0044 - accuracy: 0.998 - ETA: 53s - loss: 0.0044 - accuracy: 0.998 - ETA: 52s - loss: 0.0044 - accuracy: 0.998 - ETA: 52s - loss: 0.0044 - accuracy: 0.998 - ETA: 52s - loss: 0.0044 - accuracy: 0.998 - ETA: 52s - loss: 0.0044 - accuracy: 0.998 - ETA: 51s - loss: 0.0044 - accuracy: 0.998 - ETA: 51s - loss: 0.0044 - accuracy: 0.998 - ETA: 51s - loss: 0.0044 - accuracy: 0.998 - ETA: 51s - loss: 0.0044 - accuracy: 0.998 - ETA: 51s - loss: 0.0043 - accuracy: 0.998 - ETA: 50s - loss: 0.0043 - accuracy: 0.998 - ETA: 50s - loss: 0.0043 - accuracy: 0.998 - ETA: 50s - loss: 0.0043 - accuracy: 0.998 - ETA: 50s - loss: 0.0043 - accuracy: 0.998 - ETA: 50s - loss: 0.0043 - accuracy: 0.998 - ETA: 49s - loss: 0.0043 - acc

37800/37800 [==============================] - ETA: 14s - loss: 0.0054 - accuracy: 0.998 - ETA: 14s - loss: 0.0054 - accuracy: 0.998 - ETA: 13s - loss: 0.0054 - accuracy: 0.998 - ETA: 13s - loss: 0.0054 - accuracy: 0.998 - ETA: 13s - loss: 0.0054 - accuracy: 0.998 - ETA: 13s - loss: 0.0054 - accuracy: 0.998 - ETA: 13s - loss: 0.0054 - accuracy: 0.998 - ETA: 12s - loss: 0.0054 - accuracy: 0.998 - ETA: 12s - loss: 0.0054 - accuracy: 0.998 - ETA: 12s - loss: 0.0054 - accuracy: 0.998 - ETA: 12s - loss: 0.0054 - accuracy: 0.998 - ETA: 11s - loss: 0.0054 - accuracy: 0.998 - ETA: 11s - loss: 0.0054 - accuracy: 0.998 - ETA: 11s - loss: 0.0054 - accuracy: 0.998 - ETA: 11s - loss: 0.0054 - accuracy: 0.998 - ETA: 11s - loss: 0.0054 - accuracy: 0.998 - ETA: 10s - loss: 0.0054 - accuracy: 0.998 - ETA: 10s - loss: 0.0054 - accuracy: 0.998 - ETA: 10s - loss: 0.0054 - accuracy: 0.998 - ETA: 10s - loss: 0.0054 - accuracy: 0.998 - ETA: 10s - loss: 0.0054 - accuracy: 0.998 - ETA: 9s - loss: 0.0054 - accu

In [47]:
model.save('digit_recognizer.h5')

In [48]:
results = model.predict(test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [49]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission_mnist_datagen.csv",index=False)